In [16]:
import torch
import os
import shutil
from PIL import Image, ImageOps
import cv2
import numpy as np
from ultralytics import YOLO

In [2]:
model = YOLO("yolov8m.pt")

In [ ]:
!yolo predict model=yolov8n.pt source='Image/'

In [11]:
def loadImagesPaths(path):
    image_paths = []
    # Get a list of image file names in the directory
    image_files = [f for f in os.listdir(path) if f.endswith((".jpg", ".png", ".jpeg"))]
    for image_file in image_files:
        image_paths.append(os.path.join(path, image_file))
    return image_paths

def predictImage(image_path):
    allBirdData = []
    results = model.predict(source=image_path, save_crop=True, project="boxes", name="prediction")
    for index in range(0, len(results[0].boxes.data)):
        # IS IT A BIRD?
        if (results[0].boxes.data[index][5] == 14):
            # Extract and parse data
            tx = torch.ceil(results[0].boxes.data[index][0]).item()
            ty = torch.ceil(results[0].boxes.data[index][1]).item()
            tw = torch.ceil(results[0].boxes.data[index][2]).item()
            th = torch.ceil(results[0].boxes.data[index][3]).item()
            score = results[0].boxes.data[index][4].item()
            birdData = [tx, ty, tw, th, score]

            allBirdData.append(birdData)
    return allBirdData

In [29]:
#target_size 224/224
def cropImages(path, target_size):
    prediction_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    imageNumber = 0
    for prediction_dir in prediction_dirs:
        # Construct the path to the "crop" folder within the prediction directory
        crop_folder = os.path.join(path, prediction_dir, "crops/bird/")

        # Ensure the "crop" folder exists
        if not os.path.exists(crop_folder):
            continue  # Skip if "crop" folder is not found

        # Get a list of image files in the "crop" folder
        image_files = [f for f in os.listdir(crop_folder) if f.endswith((".jpg", ".png", ".jpeg"))]

        for image_file in image_files:
            imageNumber += 1
            # Construct the full path to the image file
            image_path = os.path.join(crop_folder, image_file)

            image = Image.open(image_path)
            old_image_height, old_image_width = image.size

            # Calculate the target size based on the larger dimension
            max_dimension = max(old_image_width, old_image_height)
            #target_size = (max_dimension, max_dimension)

            # Calculate the horizontal and vertical padding amounts
            horizontal_padding = max(0, max_dimension - old_image_width)
            vertical_padding = max(0, max_dimension - old_image_height)

            # Calculate padding parameters as (left, top, right, bottom)
            padding = (horizontal_padding // 2, vertical_padding // 2, horizontal_padding - (horizontal_padding // 2), vertical_padding - (vertical_padding // 2))

            # Pad the image
            res_image = ImageOps.expand(image, padding, fill="white")
            # Resize the padded image to the target size
            #print("Befor Lanc", image.size)
            #print("1", target_size)
            res_image = res_image.resize(target_size, Image.LANCZOS)
            #print("after Lanc", res_image.size)
            #print("2", target_size)

            # Save the image
            res_image.save(f"cropped_images/bird{imageNumber}.png")

In [5]:
def deleteimages(path):
    # Get a list of all files and subdirectories in the directory
    directory_contents = os.listdir(path)

    # Iterate through the contents and remove them
    print(directory_contents)
    for item in directory_contents:
        item_path = os.path.join(path, item)
        if os.path.isfile(item_path):
            # Remove files
            os.remove(item_path)
        elif os.path.isdir(item_path):
            # Remove directories and their contents (recursively)
            shutil.rmtree(item_path)

#deleteimages("boxes/")
#deleteimages("cropped_images/")

In [32]:
def runClassifcation():
    deleteimages("cropped_images/")
    image_paths = loadImagesPaths(path="images/")
    predictions = []
    for image in image_paths:
        predictions.append(predictImage(image))
    cropImages(path="boxes/", target_size=(224, 224))
    deleteimages("boxes/")

runClassifcation()

['bird1.png', 'bird2.png', 'bird3.png', 'bird4.png', 'bird5.png', 'bird6.png', 'bird7.png', 'bird8.png', 'bird9.png', 'bird10.png', 'bird11.png']


image 1/1 H:\Universitets-saker\D7057E\AI\YoloV8\images\fagel.jpg: 448x640 1 bird, 925.5ms
Speed: 3.0ms preprocess, 925.5ms inference, 3.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to boxes\prediction

image 1/1 H:\Universitets-saker\D7057E\AI\YoloV8\images\two_birds.jpg: 352x640 2 birds, 562.0ms
Speed: 3.0ms preprocess, 562.0ms inference, 2.0ms postprocess per image at shape (1, 3, 352, 640)
Results saved to boxes\prediction2

image 1/1 H:\Universitets-saker\D7057E\AI\YoloV8\images\Bird.jpg: 480x640 1 bird, 586.0ms
Speed: 3.0ms preprocess, 586.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to boxes\prediction3

image 1/1 H:\Universitets-saker\D7057E\AI\YoloV8\images\Bird2.jpg: 448x640 1 bird, 577.0ms
Speed: 3.0ms preprocess, 577.0ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to boxes\prediction4

image 1/1 H:\Universitets-saker\D7057E\AI\YoloV8\images\Bird3.jpg: 448x640 1 bear, 565.6ms
Speed:

Befor Lanc (356, 337)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (390, 452)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (353, 375)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (332, 273)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (750, 679)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (93, 88)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (176, 64)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (139, 157)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (251, 381)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (49, 47)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (333, 214)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
Befor Lanc (1016, 1188)
1 (224, 224)
after Lanc (224, 224)
2 (224, 224)
['prediction', 'prediction2', 'prediction3', 'prediction4', 'prediction5', 'prediction6', 'prediction7', 'prediction8', 'prediction9', 'prediction10', 'prediction1